First, we must construct our environment...

In [1]:
import numpy as np
import matplotlib
from rl_glue import RLGlue
from Agent import BaseAgent 
from Environment import BaseEnvironment  
from manager import Manager
import torch

import jdc

In [2]:
# Like shown previously, lets declare first, then implement
class CaptureGoEnvironment(BaseEnvironment):
    def env_init(self, env_info={}):
        return NotImplementedError
    
    def env_start(self):
        raise NotImplementedError

    def env_step(self, action):
        raise NotImplementedError

    def env_cleanup(self):
        raise NotImplementedError
    
    # Gets the value of a position on the game-board
    def board_index(self, x, y):
        raise NotImplementedError
    
    def inBounds(self, x, y):
        raise NotImplementedError
    
    def get_liberties(self, x, y):
        raise NotImplementedError
    
    def get_liberties_helper(self, x, y, group_color, visted_coords):
        raise NotImplementedError
    
    def did_group_captured(self, x, y):
        raise NotImplementedError
    
    # Helper: Print the board
    def print_go_board(self):
        raise NotImplementedError
    
    # Helper: Get the size of the board (how many tiles)
    def go_board_size(self):
        raise NotImplementedError
    
    # Helper: get the board (state)
    def get_board(self):
        raise NotImplementedError
    
    # Helper: get the board state in the form of a tensor
    def get_board_tensor(self): 
        raise NotImplementedError
    
    # Turns an action state into an x y pair
    # This is a litttllleee redundant since we do a bunch of transforms, but it is easier for me to read
    def action_int_to_xy(self, action_int):
        raise NotImplementedError

In [3]:
%%add_to CaptureGoEnvironment

# Create the environment, declare its variables, etc
def env_init(self, env_info={}):

    # Declare our variables
    reward = None

    # The board is an array of height x width
    # If (x,y) on the board is 0, no stone
    #   if its 1, then a white stone is there
    #   if its -1, then a black stone is there
    self.board = None
    
    termination = None

    # Which player's turn is it?
    # 0 = white's turn
    # 1 = black's turn
    self.turn = None

    # Who won?
    # 0 = nobody yet
    # 1 = white
    # -1 = black
    self.who_won = None

    self.reward_board_termination = (reward, self.board, termination)

    # Set the default board height and width
    self.board_height = env_info.get("board_height", 5) 
    self.board_width = env_info.get("board_width", 5)
    # Set the default reward per step
    self.reward_per_step = env_info.get("reward_per_step", -1)
    # Set the default winning reward and losing reward
    self.winning_reward = env_info.get("winning_reward", 100)
    self.losing_reward = env_info.get("losing_reward", -100)

In [4]:
%%add_to CaptureGoEnvironment

# Initialize the environment, set its variables, etc
# Called before an agent does anything
# Returns the initialized (full of zeros) board
def env_start(self):

    # Actually initalize
    reward = 0
    # Initialize the board, an array of 0 ints
    self.board = np.zeros(self.board_height * self.board_width, dtype=np.int8)
    termination = False
    
    self.reward_board_termination = (reward, self.board, termination)

    # We start as white's turn
    self.turn = 0
    # Nobody won yet
    self.who_won = 0

    return self.reward_board_termination[1]

In [5]:
%%add_to CaptureGoEnvironment

# Helper: Print the board
def print_go_board(self):
    x = 0
    y = 0
    
    for x in range(self.board_width):
        for y in range(self.board_height):
            pos = self.board[self.board_index(x, y)]

            stone = "."
            if pos == 1:
                stone = "w"
            if pos == -1:
                stone = "b"

            print(stone, end="")

        print("") # print a newline

# Helper: Get the size of the go board
def go_board_size(self):
    return len(self.board)

# Helper: Get the board (state)
def get_board(self):
    return self.board

# Helper: get the board state in the form of a tensor, also transforms into a float32 since thats what nn.linear expects
#   maybe in the future make it so the nn.Linear takes int8, but for now this should work
def get_board_tensor(self):
    return torch.from_numpy(self.board).to(torch.float32)

# Turns an action state into an x y pair
# This is a litttllleee redundant since we do a bunch of transforms, but it is easier for me to read
def action_int_to_xy(self, action_int):
    y = action_int // self.board_height
    x = action_int % self.board_width

    return x, y


In [6]:
# Testing
testenv = env = CaptureGoEnvironment()
testenv.env_init({"board_height" : 5, "board_width" : 5 })
testenv.env_start()

print(env.action_int_to_xy(7))

(2, 1)


In [7]:
%%add_to CaptureGoEnvironment
## Gets the index of a position on the game-board, given an x y coordinate
#   Returns the index where that position on the board is found
#   top right is x = 0 y = 0  

#   On the actual board array:
#       0 means that position is empty
#       1 means there is a white stone
#       -1 means there is a black stone
def board_index(self, x, y):
    return (self.board_width * y) + x

In [8]:
# Testing
env = CaptureGoEnvironment()
env.env_init({"board_height" : 5, "board_width" : 5 })
print(env.env_start())
print(env.board_index(3, 2))


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
13


In [ ]:
%%add_to CaptureGoEnvironment

# Returns true if the coordinate is in the board's bounds, false otherwise
def inBounds(self, x, y):
    if ((x < 0) or (y < 0)):
        return False

    if ((x >= self.board_width) or (y >= self.board_height)):
        return False

    return True

# Recursively get the liberties of a group of stones
#   returns the amount of liberties a group has, and all the coordinates we have visited
#   if group_color == 1, then we are looking for a white group
#   if group_color == -1, then we are looking for a black group
# visited_coords is a list of all coordinates we have visted, so we dont enter into an endless loop
def get_liberties_helper(self, x, y, group_color, visited_coords):
    if (x, y) in visited_coords:
        return 0, visited_coords
    visited_coords.append((x, y))

    # Check our bounds
    if not self.inBounds(x, y):
        return 0, visited_coords

    # Check what stone (or lack thereof) is on this position
    if self.board[self.board_index(x, y)] == 0:
        # liberty!
        return 1, visited_coords
    if self.board[self.board_index(x, y)] != group_color:
        # Blocked liberty
        return 0, visited_coords
    # otherwise this is part of the group, check the neighbors
    liberties = 0
    up, visited_coords = self.get_liberties_helper(x, y + 1, group_color, visited_coords)
    down, visited_coords = self.get_liberties_helper(x, y - 1, group_color, visited_coords)
    right, visited_coords = self.get_liberties_helper(x + 1, y, group_color, visited_coords)
    left, visited_coords = self.get_liberties_helper(x - 1, y, group_color, visited_coords)

    liberties = up + down + left + right
    return liberties, visited_coords

# Get the liberties of a group, starting at x, y
def get_liberties(self, x, y):
    # Set the group_color to be whatever color we started at
    liberties = 0
    coords = []
    liberties, coords = self.get_liberties_helper(x, y, self.board[self.board_index(x, y)], visited_coords=[])
    return liberties


# Returns if any group got captured, given the most recent move
#   returns 0 if no groups got captured
#   returns 1 if a white group got captured
#   returns -1 if a black group got captured
#
#   Parameters
#   x, y : The most recent move
def did_group_captured(self, x, y):
    # The (x, y) offsets we will check
    check_offsets_list = [
        [0, 0], 
        [0, 1], 
        [0, -1], 
        [1, 0], 
        [-1, 0]]

    for offset in check_offsets_list:
        this_x = x + offset[0]
        this_y = y + offset[1]

        if not self.inBounds(this_x, this_y):
            # This offset is not on the board
            continue

        # TODO: Make this respect who placed most recently, so that if you capture yourself and another stone the other stone gets captured first
        liberties_found = self.get_liberties(this_x, this_y)
        if (liberties_found == 0):
            # A group got captured, return 1 if it was a white group, -1 if it was a black group
            return self.board[self.board_index(this_x, this_y)]


    return 0 # no captures

# Take a step in the environment, actually place a stone on the board
#   Attempt to place a stone at the x, y coordinates
#   if player == self.turn, then we actually do this move, otherwise its not that players turn so we do nothing
#   if a player attempts to play an invalid move, such as out of bounds or ontop of a stone, we just ignore it and dont change the turn
# 
#   Returns the reward_board_termination tuple that represents the changed environment
def env_step(self, x, y, player):
    # Check if the game has already finished or not
    if (self.who_won != 0):
        return None

    if (player != self.turn):
        # Its not this player's turn
        reward = 0
        term = self.reward_board_termination[2]
        self.reward_board_termination = (reward, self.board, term)
        return self.reward_board_termination

    # Check that the move is in bounds
    if not self.inBounds(x, y):
        # Just pretend like this turn didn't happen, but take away some points
        reward = -5 # you aren't following the rules!
        term = self.reward_board_termination[2]
        self.reward_board_termination = (reward, self.board, term)
        return self.reward_board_termination

    # Check that the position is empty
    if (self.board[self.board_index(x, y)] != 0):
        # Just pretend like this turn didn't happen, but take away some points
        reward = -5 # you aren't following the rules!
        term = self.reward_board_termination[2]
        self.reward_board_termination = (reward, self.board, term)
        return self.reward_board_termination

    # If it's white's turn, stone = 1
    #   if it's black's turn, stone = -1
    stone = 0
    if (self.turn == 0):
        stone = 1
    else:
        stone = -1

    # Place the stone on the board
    self.board[self.board_index(x, y)] = stone


    # Get the reward
    reward = self.reward_per_step

    # Most complicated part: check if anything was captured, and therefore if we need to terminate or not
    termination = False
    captured_group = self.did_group_captured(x, y)
    if (captured_group == 1):
        # white got captured
        if (self.turn == 1): # If it is white's turn right now, we are giving white the reward
            reward += self.losing_reward
        else:
            reward += self.winning_reward
        
        termination = True
    elif (captured_group == -1):
        # black got captured
        if (self.turn == 1): # If it is white's turn right now, we are giving white the reward
            reward += self.winning_reward
        else:
            reward += self.losing_reward

        termination = True

    
    # The winner was who did not get captured, or nobody if nothing was captured
    self.who_won = captured_group * -1
    # It is now the other player's turn
    self.turn = not self.turn
    self.reward_board_termination = (reward, self.board, termination)

    return self.reward_board_termination





In [10]:
# Testing

# Testing
env = CaptureGoEnvironment()
env.env_init({"board_height" : 5, "board_width" : 5 })
print(env.env_start())

white = 0
black = 1
env.env_step(3, 2, white)
env.env_step(3, 4, black)
env.env_step(1, 1, white)
env.env_step(0, 0, black)
env.env_step(4, 4, white)
print(env.env_step(4, 3, black))

env.print_go_board()

print("\n=============\n")
env = CaptureGoEnvironment()
env.env_init({"board_height" : 5, "board_width" : 5 })
env.env_start()

env.board = [0, 0, 0, 0, 0, 
             0, 0,  1,  1,   1,
             0, 0, -1, -1,  -1,
             0, 0,  1,  1,  -1,
             0, 0,  0,  1,  -1,]
env.print_go_board()
print(env.env_step(1, 2, white))
env.print_go_board()
print(env.go_board_size())


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(-101, array([-1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,
        0,  0, -1,  0,  0,  0, -1,  1], dtype=int8), True)
b....
.w...
.....
..w.b
...bw


.....
.....
.wbw.
.wbww
.wbbb
(-101, [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, -1, -1, -1, 0, 0, 1, 1, -1, 0, 0, 0, 1, -1], True)
.....
..w..
.wbw.
.wbww
.wbbb
25


In [11]:
%%add_to CaptureGoEnvironment

def env_cleanup(self):
    reward = 0
    # Initialize the board, an array of 0 ints
    self.board = np.zeros(self.board_height * self.board_width, dtype=np.int8)
    termination = False
    
    self.reward_board_termination = (reward, self.board, termination)

    # We start as white's turn
    self.turn = 0
    # Nobody won yet
    self.who_won = 0
    

Now, lets implement an agent that can learn
I used the pytorch deep-q learning tutorial to help me with this https://docs.pytorch.org/tutorials/intermediate/reinforcement_q_learning.html
Deep-q learning is how google created it's super strong go neural net, so I think I might have some success

We say that the action an agent can take is placing a stone on a tile, so we have as many actions as tiles

In [12]:
import math
import random

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as functional

from collections import deque

Create the replay memory, used https://docs.pytorch.org/tutorials/intermediate/reinforcement_q_learning.html to help


In [13]:
# Our replay memory for the deep-q neuralNet
class DeepQReplayMemory(object):
    # Create the replay memory object, capacity is how much memory we store
    # The memory replay object is essentially just a queue that stores memory
    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    # Push a single observation into the queue
    def push(self, state, action, state_prime, reward):
        self.memory.append((state, action, state_prime, reward)) # append this observation to our memory

    # Sample a random observation from the replay memory of sample_size
    def sample(self, sample_size):
        return random.sample(self.memory, sample_size)
    
    # Get how much memory we have
    def __len__(self):
        return len(self.memory)

Create the actual deep-q neural network, mostly relies on pytorch to do the dirty work.
Again, following the deep-q tutorial for pytorch https://docs.pytorch.org/tutorials/intermediate/reinforcement_q_learning.html

In [14]:
# The deep-q network for our go game
class goDQN(nn.Module):
    
    def __init__(self, num_inputs, hidden_layer_1_size, hidden_layer_2_size, num_outputs):
        super(goDQN, self).__init__()

        # Create the layers, I actually want multiple hidden layers
        self.input_layer = nn.Linear(num_inputs, hidden_layer_1_size) # Create the input layer
        self.hidden_layer_1 = nn.Linear(hidden_layer_1_size, hidden_layer_2_size) # Create the first hidden layer
        self.hidden_layer_2 = nn.Linear(hidden_layer_2_size, hidden_layer_2_size) # Create the second hidden layer
        self.output_layer = nn.Linear(hidden_layer_2_size, num_outputs)
    
    # Process an input through the neural net, return what we got as our output
    # Returns a tensor representing the output layer's output. A tensor is essentially just a multidimensional matrix with a single datatype
    def forward(self, input):
        process = input
        # Run the input through our layers
        process = functional.relu(self.input_layer(process)) # We call relu to make sure we dont pass a negative number into future layers
        process = functional.relu(self.hidden_layer_1(process))
        process = functional.relu(self.hidden_layer_2(process))
        # Return our output
        return self.output_layer(process)

Create the training functions for our go agents, we have a black agent and a white agent for each player of go

In [15]:
HIDDEN_LAYER_1_SIZE = 128
HIDDEN_LAYER_2_SIZE = 64
REPLAY_MEMORY_SIZE = 1000

# Our epsilon for epsilon_greedy
EPSILON = 0.1
# Our stepsize / learning rate 
STEP_SIZE = 0.01
# Our discount factor
GAMMA = 0.9

goEnv = CaptureGoEnvironment()
goEnv.env_init({"board_height" : 5, "board_width" : 5 })
goEnv.env_start() # So that the board is actually created and we can grab its size

white_agent = goDQN(goEnv.go_board_size(), HIDDEN_LAYER_1_SIZE, HIDDEN_LAYER_2_SIZE, goEnv.go_board_size())
black_agent = goDQN(goEnv.go_board_size(), HIDDEN_LAYER_1_SIZE, HIDDEN_LAYER_2_SIZE, goEnv.go_board_size())

# Create the target networks for each of our agents
white_agent_target_network = goDQN(goEnv.go_board_size(), HIDDEN_LAYER_1_SIZE, HIDDEN_LAYER_2_SIZE, goEnv.go_board_size())
black_agent_target_network = goDQN(goEnv.go_board_size(), HIDDEN_LAYER_1_SIZE, HIDDEN_LAYER_2_SIZE, goEnv.go_board_size())

white_memory = DeepQReplayMemory(REPLAY_MEMORY_SIZE)
black_memory = DeepQReplayMemory(REPLAY_MEMORY_SIZE)

# Create our pytorch optimizer, we have to choose an optimizer, it might be worth trying multiple optimizers in the future
#   for now we will go with what the tutorial https://docs.pytorch.org/tutorials/intermediate/reinforcement_q_learning.html
#   says to use, AdamW
white_optimizer = optim.AdamW(white_agent.parameters(), lr=STEP_SIZE)
black_optimizer = optim.AdamW(black_agent.parameters(), lr=STEP_SIZE)


def select_action(agent, board_state):
    # Follow epsilon to choose a random state or consult our learned Q
    rand_value = random.random() # Get a random number between 0-1
    if (rand_value < EPSILON):
        # Choose a random action
        return torch.tensor([[random.randint(0, goEnv.go_board_size() - 1)]])
    else:
        # Return the output node with the highest signal, the index of that is our action
        #   ie what space we will attempt to place a piece on
        #print(board_state)
        #print(agent(board_state))

        # Get the output with the highest signal as the action to take
        agent_outputs = agent(board_state)
        max_value = agent_outputs.max()
        for i in range(len(agent_outputs)):
            if agent_outputs[i] == max_value:
                return torch.tensor([[i]])
        raise IndexError # For some reason we could not find it?
        #return agent(board_state).max().indices.view(1, 1)
    


Actually train our agents, again using the pytorch deep-q network tutorial https://docs.pytorch.org/tutorials/intermediate/reinforcement_q_learning.html
Also used https://www.geeksforgeeks.org/deep-learning/implementing-deep-q-learning-using-tensorflow/ as a tutorial for implementing this

This is a deep-q network, so we have a target network and our actual agent networks 
The target network is only updated every 1000-10000 steps, so that we maintain stability

In [16]:
BATCH_SIZE = 100

# This part heavily relies on the pytorch tutorial for deep-q learning
# https://docs.pytorch.org/tutorials/intermediate/reinforcement_q_learning.html
def optimize_model(agent_policy_network, agent_target_network, agent_replay_memory, agent_optimizer):
    if len(agent_replay_memory < BATCH_SIZE):
        return # we dont have enough replay memory to actually create a full batch

    # This next bit is copied directly from the tutorial, its a complicated part that turns the replay memory into
    # a batch that we can actually use to learn with
    transitions = agent_replay_memory.sample(BATCH_SIZE)
    states, actions, states_prime, rewards = zip(*transitions) # got from https://www.geeksforgeeks.org/deep-learning/implementing-deep-q-learning-using-tensorflow/

    state_batch = torch.cat(states)
    action_batch = torch.cat(actions)
    reward_batch = torch.cat(actions)

    # Get our Q-value for each state, action pair
    state_action_Q_values = agent_policy_network(state_batch).gather(1, action_batch)

    # Compute the expected value for each state_prime
    state_prime_expected_values = torch.zeros(BATCH_SIZE) # Initialize theh array
    i = 0 # Counter for which element in the batches we are in
    for state_prime in states_prime:
        this_expected_value = agent_target_network(state_prime).max(1) # Get the output from the TARGET network
        this_expected_value = (this_expected_value * GAMMA) + reward_batch[i]

        state_prime_expected_values[i] = this_expected_value

        i += 1
        

    # Compute loss, unlike the tutorial I would rather use MSE loss since that is what we used in class
    criterion = nn.MSELoss()
    losses = criterion(state_action_Q_values, state_prime_expected_values.unsqueeze(1)) # Unsqueeze(1) makes each tensor element a seperate array in a tenso

    # optimize our network, ie update the parameters of our policy's network
    agent_optimizer.zero_grad() # This resets the gradients of our optimizer
    losses.backward() # God function which will actually derive our parameter's sums and
    # Here the tutorial clips the gradient values, Im not gonna do that for now to see what happens
    agent_optimizer.step() # Actually update the parameters!



Next step: Actually implementing the training loop

In [17]:
# How many episodes would we like to run?
NUM_EPISODES = 10

for episode in range(NUM_EPISODES):
    # Prepare the environment
    goEnv.env_cleanup()
    goEnv.env_start()

    while (goEnv.who_won == 0): # While the game is still going, and nobody won
        # Swap who runs each time, since we have two different agents playing against eachother
        current_agent = white_agent
        current_agent_target_network = white_agent_target_network
        current_agent_replay_memory = white_memory
        current_agent_optimizer = white_optimizer
        if (goEnv.turn == 1): # its actually black's turn
            current_agent = black_agent
            current_agent_target_network = black_agent_target_network
            current_agent_replay_memory = black_memory
            current_agent_optimizer = black_optimizer

        # select our action
        action = select_action(current_agent, goEnv.get_board_tensor())
        
        # Act on this action
        this_x, this_y = goEnv.action_int_to_xy(action)
        reward_state_term = goEnv.env_step(this_x, this_y, goEnv.turn)
        print(goEnv.print_go_board())
        print(action, this_x, this_y, reward_state_term)




.....
.....
.w...
.....
.....
None
tensor([[7]]) tensor([[2]]) tensor([[1]]) (-1, array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0], dtype=int8), False)
.....
.....
.w...
...b.
.....
None
tensor([[18]]) tensor([[3]]) tensor([[3]]) (-1, array([ 0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0, -1,  0,  0,  0,  0,  0,  0], dtype=int8), False)


TypeError: 'tuple' object does not support item assignment